# Debugging Block Fetching Logic

This notebook isolates the `BlockHeightCollector` class from `src.demurrage_distribution` to debug the process of finding the latest outgoing transaction and the subsequent blocks.

In [1]:
import os
import sys
import logging
import time
from decimal import Decimal
from dotenv import load_dotenv

# Add src directory to path to allow importing
# Assumes the notebook is in the project root, and src is a subdirectory
module_path = os.path.abspath(os.path.join(os.getcwd(), 'src')) 
if module_path not in sys.path:
    sys.path.insert(0, module_path) # Prepend src path

# Load environment variables from .env file in the current directory (project root)
dotenv_path = os.path.join(os.getcwd(), '.env') 
load_dotenv(dotenv_path=dotenv_path)

# --- Setup Logging --- 
# Remove existing handlers to avoid duplicate logs in notebooks
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
# Configure logging
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', 
                    stream=sys.stdout) # Log to stdout for notebook visibility

# Silence overly verbose libraries if needed
logging.getLogger("urllib3").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)
logger.info("Logging configured")

# --- Import the Class --- 
try:
    # Since src is now in path, we can import directly
    from demurrage_distribution import BlockHeightCollector
    logger.info("Successfully imported BlockHeightCollector")
except ImportError as e:
    logger.error(f"Failed to import BlockHeightCollector: {e}")
    logger.error(f"Current sys.path: {sys.path}")
    logger.error("Make sure 'src' is in the path and demurrage_distribution.py exists.")
except Exception as e:
    logger.error(f"An unexpected error occurred during import: {e}")

2025-05-01 08:28:57,268 - __main__ - INFO - Logging configured
2025-05-01 08:28:57,302 - __main__ - INFO - Successfully imported BlockHeightCollector


## Step 1: Instantiate the Collector

This will load the wallet address from the environment variable `WALLET_ADDRESS`.

In [2]:
wallet_addr = os.getenv('WALLET_ADDRESS')
if not wallet_addr:
    logger.warning('WALLET_ADDRESS environment variable not found!')
    # You might want to hardcode it here for testing if needed
    wallet_addr = "9fE5o7913CKKe6wvNgM11vULjTuKiopPcvCaj7t2zcJWXM2gcLu" 
    logger.warning(f'Using hardcoded address for testing: {wallet_addr}')

collector = None # Initialize to None
try:
    # Check if BlockHeightCollector was imported successfully before trying to instantiate
    if 'BlockHeightCollector' in locals() or 'BlockHeightCollector' in globals():
        collector = BlockHeightCollector(wallet_address=wallet_addr)
        logger.info(f"BlockHeightCollector instantiated for address: {collector.wallet_address}")
    else:
        logger.error("BlockHeightCollector class not available due to import error.")
except Exception as e:
    logger.error(f"Error instantiating BlockHeightCollector: {e}", exc_info=True)

2025-05-01 08:28:57,307 - __main__ - INFO - BlockHeightCollector instantiated for address: 9fE5o7913CKKe6wvNgM11vULjTuKiopPcvCaj7t2zcJWXM2gcLu


## Step 2: Find Latest Outgoing Block Height

Call `get_latest_outgoing_block()` to find the inclusion height of the most recent transaction sent *from* the wallet address. We expect this to use the updated code with `limit=200` and debug logging.

In [3]:
latest_outgoing_height = 0 # Default value
if collector:
    logger.info("--- Calling get_latest_outgoing_block --- ")
    try:
        latest_outgoing_height = collector.get_latest_outgoing_block()
        logger.info(f"--- get_latest_outgoing_block finished --- ")
        print(f'Latest outgoing block height found: {latest_outgoing_height}')
    except Exception as e:
        logger.error(f"Error calling get_latest_outgoing_block: {e}", exc_info=True)
else:
    logger.error("Collector object not instantiated, cannot call method.")

2025-05-01 08:28:57,329 - __main__ - INFO - --- Calling get_latest_outgoing_block --- 
2025-05-01 08:28:57,330 - demurrage_distribution - INFO - Fetching transactions for address: 9fE5o7913CKKe6wvNgM11vULjTuKiopPcvCaj7t2zcJWXM2gcLu
2025-05-01 08:29:04,191 - demurrage_distribution - DEBUG - Checking initial 200 transactions...
2025-05-01 08:29:04,192 - demurrage_distribution - DEBUG -   Checking tx at height: 1515496
2025-05-01 08:29:04,192 - demurrage_distribution - DEBUG -   Checking tx at height: 1515256
2025-05-01 08:29:04,192 - demurrage_distribution - DEBUG -   Checking tx at height: 1515231
2025-05-01 08:29:04,192 - demurrage_distribution - DEBUG -   Checking tx at height: 1515205
2025-05-01 08:29:04,193 - demurrage_distribution - DEBUG -   Checking tx at height: 1514974
2025-05-01 08:29:04,193 - demurrage_distribution - DEBUG -   Checking tx at height: 1514857
2025-05-01 08:29:04,193 - demurrage_distribution - DEBUG -   Checking tx at height: 1514457
2025-05-01 08:29:04,193 - de

## Step 3: Get Blocks Since Last Outgoing

Call `get_blocks_since_last_outgoing()`. This relies on the result from the previous step. If the previous step returned 0, this should return an empty list.

In [4]:
blocks = [] # Default value
if collector:
    logger.info("--- Calling get_blocks_since_last_outgoing --- ")
    try:
        # We can optionally override the min_block here for testing if needed
        # collector.min_block_override = 123456 # Example override
        
        blocks = collector.get_blocks_since_last_outgoing()
        
        # Clean up override if set
        # if hasattr(collector, 'min_block_override'):
        #     del collector.min_block_override
            
        logger.info(f"--- get_blocks_since_last_outgoing finished --- ")
        print(f'Found {len(blocks)} blocks since last outgoing transaction:')
        # Print first and last few blocks if the list is long
        if len(blocks) > 10:
            print(f'  {blocks[:5]} ... {blocks[-5:]}')
        else:
            print(f'  {blocks}')
    except Exception as e:
        logger.error(f"Error calling get_blocks_since_last_outgoing: {e}", exc_info=True)
else:
     logger.error("Collector object not instantiated, cannot call method.")

2025-05-01 08:29:04,218 - __main__ - INFO - --- Calling get_blocks_since_last_outgoing --- 
2025-05-01 08:29:04,219 - demurrage_distribution - INFO - Fetching transactions for address: 9fE5o7913CKKe6wvNgM11vULjTuKiopPcvCaj7t2zcJWXM2gcLu
2025-05-01 08:29:21,026 - demurrage_distribution - DEBUG - Checking initial 200 transactions...
2025-05-01 08:29:21,027 - demurrage_distribution - DEBUG -   Checking tx at height: 1515496
2025-05-01 08:29:21,027 - demurrage_distribution - DEBUG -   Checking tx at height: 1515256
2025-05-01 08:29:21,027 - demurrage_distribution - DEBUG -   Checking tx at height: 1515231
2025-05-01 08:29:21,028 - demurrage_distribution - DEBUG -   Checking tx at height: 1515205
2025-05-01 08:29:21,028 - demurrage_distribution - DEBUG -   Checking tx at height: 1514974
2025-05-01 08:29:21,029 - demurrage_distribution - DEBUG -   Checking tx at height: 1514857
2025-05-01 08:29:21,029 - demurrage_distribution - DEBUG -   Checking tx at height: 1514457
2025-05-01 08:29:21,030

In [5]:
logger.info("--- Step 4: Fetch Miners & Generate Distribution ---")
import json
# --- Configuration for Distribution ---
# Get from environment or use defaults
# Use realistic defaults based on your script if ENV vars aren't set in notebook context
TOTAL_DISTRIBUTION_AMOUNT = Decimal(os.getenv('TOTAL_DISTRIBUTION_AMOUNT', '67.6767'))
TOKEN_NAME = os.getenv('TOKEN_NAME', 'Ergonaut Demurrage Token')
POOL_FEE_PERCENTAGE = Decimal(os.getenv('POOL_FEE_PERCENTAGE', '0.01')) # 1%
POOL_FEE_ADDRESS = os.getenv('POOL_FEE_ADDRESS', '9iAFh6SzzSbowjsJPaRQwJfx4Ts4EzXt78UVGLgGaYTdab8SiEt')

# Calculate pool fee amount and miner share
pool_fee_amount = (TOTAL_DISTRIBUTION_AMOUNT * POOL_FEE_PERCENTAGE).quantize(Decimal('1E-8')) # NanoErg precision
actual_miner_distribution_amount = (TOTAL_DISTRIBUTION_AMOUNT - pool_fee_amount).quantize(Decimal('1E-8'))

logger.info(f'Total Dist: {TOTAL_DISTRIBUTION_AMOUNT}, Pool Fee ({POOL_FEE_PERCENTAGE*100}%): {pool_fee_amount}, Miner Share: {actual_miner_distribution_amount}')

distribution_json = None # Default value
miners_data = None # Default value

if collector and blocks: # Only proceed if collector exists and blocks were found
    logger.info(f"--- Calling fetch_miners_data for {len(blocks)} blocks ---")
    try:
        miners_data = collector.fetch_miners_data(blocks)
        logger.info(f"--- fetch_miners_data finished ---")

        if miners_data and miners_data.get('miners') is not None: # Check if 'miners' key exists
            logger.info(f"Fetched data for {len(miners_data.get('miners', []))} miners.")

            # Proceed even if miner list is empty, generate_distribution handles it
            logger.info("--- Calling generate_distribution --- ")
            distribution_json = collector.generate_distribution(
                miners_data=miners_data,
                miner_distribution_amount=actual_miner_distribution_amount, # Use calculated amount
                token_name=TOKEN_NAME,
                pool_fee_amount=pool_fee_amount, # Use calculated fee amount
                pool_fee_address=POOL_FEE_ADDRESS
            )
            logger.info("--- generate_distribution finished --- ")

            print("\nGenerated Distribution JSON:\n")
            print(json.dumps(distribution_json, indent=2))

        else:
            logger.warning("Miners data API did not return expected format or 'miners' key. Cannot generate distribution.")
            print(f"Received miners data: {miners_data}") # Print what was received

    except Exception as e:
        logger.error(f"Error during miners data fetching or distribution generation: {e}", exc_info=True)
elif not collector:
    logger.error("Collector object not instantiated, cannot fetch miners data or generate distribution.")
else: # blocks is empty
    logger.warning("Block list is empty. Skipping miners data fetch and distribution generation.")


2025-05-01 08:29:27,683 - __main__ - INFO - --- Step 4: Fetch Miners & Generate Distribution ---
2025-05-01 08:29:27,685 - __main__ - INFO - Total Dist: 67.6767, Pool Fee (1.00%): 0.67676700, Miner Share: 66.99993300
2025-05-01 08:29:27,685 - __main__ - INFO - --- Calling fetch_miners_data for 54 blocks ---
2025-05-01 08:29:27,686 - demurrage_distribution - INFO - Fetching miners data for 54 blocks using https://api.ergominers.com/sigscore/miners/average-participation
2025-05-01 08:29:29,712 - demurrage_distribution - INFO - Found 62 miners in response
2025-05-01 08:29:29,714 - __main__ - INFO - --- fetch_miners_data finished ---
2025-05-01 08:29:29,715 - __main__ - INFO - Fetched data for 62 miners.
2025-05-01 08:29:29,715 - __main__ - INFO - --- Calling generate_distribution --- 
2025-05-01 08:29:29,716 - demurrage_distribution - INFO - Total participation percentage from miners data: 112.6538
2025-05-01 08:29:29,716 - demurrage_distribution - DEBUG - Checking condition to add pool f

## Analysis

Examine the logs and output above.
1. Did the `get_latest_outgoing_block` call show debug logs with `Checking tx at height: ...`?
2. Did the initial API call log (if visible through requests logging) show `limit=200`?
3. Was a non-zero `latest_outgoing_block_height` found?
4. Did `get_blocks_since_last_outgoing` return a list of block heights?